<a href="https://colab.research.google.com/github/riotrip/ml-smt5/blob/main/TG1_2_JS07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Rio Tri Prayogo - 2341720236 - TI 3F/25**
---
## **JS07 - Approximate Nearest Neighbors (ANN)**

### **Tugas 1**
Lakukan percobaan pada metric distance yang berbeda, 1000 vs 1.000.000 data, 2D vs 5D data, untuk algoritma,

a. ANNOY

b. FAISS

c. HNSW

Catat performansinya dalam bentuk tabel, misa

In [1]:
# Install ANNOY
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp312-cp312-linux_x86_64.whl size=551807 sha256=21f5cbf1c15042fb7386faf545f4e3cbd84b4f40f61318eef40c095da2ce782e
  Stored in directory: /root/.cache/pip/wheels/db/b9/53/a3b2d1fe1743abadddec6aa541294b24fdbc39d7800bc57311
Successfully built annoy


In [2]:
# Install FAISS
!pip install faiss-cpu
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 68.5 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [3]:
# Install HNSW
!pip install hnswlib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp312-cp312-linux_x86_64.whl size=2528146 sha256=607c72f4ee5f1024bd4b3045d487015b8eba69d0c270a50b07a93bd2725f6d28
  Stored in directory: /root/.cache/pip/wheels/ac/39/b3/cbd7f9cbb76501d2d5fbc84956e70d0b94e788aac87bda465e
Successfully built hnswlib


In [ ]:
import numpy as np
import time
from annoy import AnnoyIndex
import faiss
import hnswlib
import pandas as pd

def benchmark_annoy(X, query, k, dim, n_trees=10):
    """Benchmark Annoy dengan Euclidean distance"""
    ann_index = AnnoyIndex(dim, 'euclidean')

    start = time.time()
    for i in range(len(X)):
        ann_index.add_item(i, X[i])
    ann_index.build(n_trees)
    build_time = time.time() - start

    start = time.time()
    neighbors = ann_index.get_nns_by_vector(query[0], k, include_distances=True)
    query_time = time.time() - start

    return build_time, query_time, neighbors[0][:5]

def benchmark_faiss(X, query, k, dim):
    """Benchmark FAISS dengan L2 distance"""
    faiss_index = faiss.IndexFlatL2(dim)

    start = time.time()
    faiss_index.add(X)
    build_time = time.time() - start

    start = time.time()
    distances, indices = faiss_index.search(query, k)
    query_time = time.time() - start

    return build_time, query_time, indices[0][:5]

def benchmark_hnsw(X, query, k, dim, ef_construction=200, M=16, ef=50):
    """Benchmark HNSW dengan L2 distance"""
    hnsw_index = hnswlib.Index(space='l2', dim=dim)

    start = time.time()
    hnsw_index.init_index(max_elements=len(X), ef_construction=ef_construction, M=M)
    hnsw_index.add_items(X)
    build_time = time.time() - start

    hnsw_index.set_ef(ef)

    start = time.time()
    labels, distances = hnsw_index.knn_query(query, k=k)
    query_time = time.time() - start

    return build_time, query_time, labels[0][:5]

# Konfigurasi eksperimen
configs = [
    (1000, 2),    # 1000 data, 2D
    (1000, 5),    # 1000 data, 5D
    (1000000, 2), # 1M data, 2D
    (1000000, 5), # 1M data, 5D
]

k = 10
results = []

print("=== EKSPERIMEN PERBANDINGAN PERFORMANCE ANN, FAISS, HNSW ===\n")

for n_data, dim in configs:
    print(f"Testing: {n_data:,} data, {dim}D")

    # Generate data
    np.random.seed(42)  # Untuk reproducibility
    X = np.random.random((n_data, dim)).astype(np.float32)
    query = np.random.random((1, dim)).astype(np.float32)

    # Benchmark masing-masing algoritma
    print("  Running Annoy...")
    annoy_build, annoy_query, annoy_neighbors = benchmark_annoy(X, query, k, dim)

    print("  Running FAISS...")
    faiss_build, faiss_query, faiss_neighbors = benchmark_faiss(X, query, k, dim)

    print("  Running HNSW...")
    hnsw_build, hnsw_query, hnsw_neighbors = benchmark_hnsw(X, query, k, dim)

    # Simpan hasil
    results.append({
        'config': f"{n_data:,}/{dim}D",
        'annoy_build': annoy_build,
        'annoy_query': annoy_query,
        'faiss_build': faiss_build,
        'faiss_query': faiss_query,
        'hnsw_build': hnsw_build,
        'hnsw_query': hnsw_query,
        'annoy_neighbors': annoy_neighbors,
        'faiss_neighbors': faiss_neighbors,
        'hnsw_neighbors': hnsw_neighbors
    })

    print(f"  Completed: {n_data:,} data, {dim}D\n")

# Buat tabel hasil
df_results = pd.DataFrame(results)
print("="*80)
print("HASIL PERBANDINGAN PERFORMANCE")
print("="*80)

# Tabel Build Time
print("\nBUILD TIME (detik):")
build_table = pd.DataFrame({
    'Jumlah Data/Dimensi': df_results['config'],
    'ANNOY': [f"{x:.4f}s" for x in df_results['annoy_build']],
    'FAISS': [f"{x:.4f}s" for x in df_results['faiss_build']],
    'HNSW': [f"{x:.4f}s" for x in df_results['hnsw_build']]
})
print(build_table.to_string(index=False))

# Tabel Query Time
print("\nQUERY TIME (detik):")
query_table = pd.DataFrame({
    'Jumlah Data/Dimensi': df_results['config'],
    'ANNOY': [f"{x:.6f}s" for x in df_results['annoy_query']],
    'FAISS': [f"{x:.6f}s" for x in df_results['faiss_query']],
    'HNSW': [f"{x:.6f}s" for x in df_results['hnsw_query']]
})
print(query_table.to_string(index=False))

# Analisis tambahan
print("\n" + "="*80)
print("ANALISIS:")
print("="*80)

for i, result in enumerate(results):
    print(f"\nKonfigurasi: {result['config']}")
    print(f"Build Time Tercepat: {min(result['annoy_build'], result['faiss_build'], result['hnsw_build']):.4f}s")
    print(f"Query Time Tercepat: {min(result['annoy_query'], result['faiss_query'], result['hnsw_query']):.6f}s")

    # Cek konsistensi hasil (5 neighbor teratas)
    print("5 Neighbor Teratas:")
    print(f"  Annoy: {result['annoy_neighbors']}")
    print(f"  FAISS: {result['faiss_neighbors']}")
    print(f"  HNSW:  {result['hnsw_neighbors']}")

=== EKSPERIMEN PERBANDINGAN PERFORMANCE ANN, FAISS, HNSW ===

Testing: 1,000 data, 2D
  Running Annoy...
  Running FAISS...
  Running HNSW...
  Completed: 1,000 data, 2D

Testing: 1,000 data, 5D
  Running Annoy...
  Running FAISS...
  Running HNSW...
  Completed: 1,000 data, 5D

Testing: 1,000,000 data, 2D
  Running Annoy...
  Running FAISS...
  Running HNSW...
  Completed: 1,000,000 data, 2D

Testing: 1,000,000 data, 5D
  Running Annoy...
  Running FAISS...
  Running HNSW...
  Completed: 1,000,000 data, 5D

HASIL PERBANDINGAN PERFORMANCE

BUILD TIME (detik):
Jumlah Data/Dimensi    ANNOY   FAISS      HNSW
           1,000/2D  0.0214s 0.0001s   0.0468s
           1,000/5D  0.0144s 0.0000s   0.0537s
       1,000,000/2D 27.6529s 0.0026s 102.4016s
       1,000,000/5D 22.9693s 0.0146s 169.9104s

QUERY TIME (detik):
Jumlah Data/Dimensi     ANNOY     FAISS      HNSW
           1,000/2D 0.000079s 0.000052s 0.000057s
           1,000/5D 0.000058s 0.000043s 0.000057s
       1,000,000/2D 0.000106

### **Tugas 2**

Lakukan percobaan penggunaan ANNOY, FAISS, dan HNSWLIB pada dataset sekunder berukuran besar (Micro Spotify) pada link berikut: https://www.kaggle.com/datasets/bwandowando/spotify-songs-with-attributes-and-lyrics/data .

Download data dan load CSV filenya (pilih dataset yg pertama dari dua dataset).
* Pilih hanya fitur numerik saja, dan lakukan normalisasi menggunakan StandardScaler.
* Lakukan pencarian track terdekat dan bandingkan hasilnya.
* Lakkan perbandingan pada exact NN, ANNOY, FAISS, dan HNSW

Install kagglehub[panda-datasets]

In [17]:
!pip install kagglehub[pandas-datasets]

Boilerplate

In [24]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "songs_with_attributes_and_lyrics.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "bwandowando/spotify-songs-with-attributes-and-lyrics",
  file_path,
)

print("First 5 records:", df.head())

/tmp/ipython-input-797073249.py:10: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'spotify-songs-with-attributes-and-lyrics' dataset.
First 5 records:                        id             name  \
0  0Prct5TDjAnEgIqbxcldY9                !   
1  2ASl4wirkeYm3OWZxXKYuq               !!   
2  69lcggVPmOr9cvPx9kLiiN  !!! - Interlude   
3  4U7dlZjg1s9pjdppqZy0fm   !!De Repente!!   
4  4v1IBp3Y3rpkWmWzIlkYju   !!De Repente!!   

                               album_name       artists  danceability  energy  \
0                              UNDEN!ABLE  ['HELLYEAH']         0.415  0.6050   
1                                     NaN       Yxngxr1         0.788  0.6480   
2                       Where I Belong EP    ['Glowie']         0.000  0.0354   
3  Un Palo Al Agua (20 Grandes Canciones)   ['Rosendo']         0.657  0.8820   
4                          Fuera De Lugar   ['Rosendo']         0.659  0.8930   

  key  loudness mode  speechiness  acousticness  instrumentalness  liveness  \
0   7   -11.157    1       0.0575       0.001

In [22]:
features = ['danceability', 'energy', 'loudness', 'speechiness',
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
X = df[features].values

# Normalisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k = 10

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955320 entries, 0 to 955319
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                955320 non-null  object 
 1   name              955309 non-null  object 
 2   album_name        385557 non-null  object 
 3   artists           955318 non-null  object 
 4   danceability      955320 non-null  float64
 5   energy            955320 non-null  float64
 6   key               955320 non-null  object 
 7   loudness          955320 non-null  float64
 8   mode              955320 non-null  object 
 9   speechiness       955320 non-null  float64
 10  acousticness      955320 non-null  float64
 11  instrumentalness  955320 non-null  float64
 12  liveness          955320 non-null  float64
 13  valence           955320 non-null  float64
 14  tempo             955320 non-null  float64
 15  duration_ms       955320 non-null  float64
 16  lyrics            95

Split data menjadi corpus dan query

In [25]:
# split data menjadi query dan corpus
np.random.seed(42)
n_samples = X_scaled.shape[0]
n_queries = 95
query_indices = np.random.choice(n_samples, n_queries, replace=False)
queries = X_scaled[query_indices]
corpus = X_scaled

Membuat fungsi hitung recall

In [26]:
def recall_at_k(true_idx, pred_idx, k):
    recalls = []
    for i in range(len(true_idx)):
        true_set = set(true_idx[i, :k])
        pred_set = set(pred_idx[i, :k])
        recalls.append(len(true_set & pred_set) / k)
    return np.mean(recalls)

Perbandingan pada exact NN, ANNOY, FAISS, dan HNSW

In [27]:
# -------------------------------
# Exact Nearest Neighbors (NN)
# -------------------------------
print("=== Exact Nearest Neighbors (NN) ===")
t0 = time.time()
nn = NearestNeighbors(n_neighbors=k, metric='euclidean')
nn.fit(corpus)
build_time = time.time() - t0

t0 = time.time()
dist_exact, idx_exact = nn.kneighbors(queries)
query_time = time.time() - t0
print(f"Build time: {build_time:.4f} s, Query time: {query_time:.4f} s")


# -------------------------------
# Annoy
# -------------------------------
print("\n=== Annoy ===")
dims = corpus.shape[1]
annoy_idx = AnnoyIndex(dims, 'euclidean')

t0 = time.time()
for i, vec in enumerate(corpus):
    annoy_idx.add_item(i, vec)
annoy_idx.build(50) # jumlah tree
build_time_annoy = time.time() - t0

t0 = time.time()
idx_annoy = []
for q in queries:
    idx_annoy.append(annoy_idx.get_nns_by_vector(q, k))
query_time_annoy = time.time() - t0
idx_annoy = np.array(idx_annoy)
print(f"Build time: {build_time_annoy:.4f} s, Query time: {query_time_annoy:.4f} s")

# -------------------------------
# FAISS
# -------------------------------
print("\n=== FAISS ===")
d = corpus.shape[1]
index_faiss = faiss.IndexFlatL2(d)
t0 = time.time()
index_faiss.add(corpus.astype(np.float32))
build_time_faiss = time.time() - t0

t0 = time.time()
dist_faiss, idx_faiss = index_faiss.search(queries.astype(np.float32), k)
query_time_faiss = time.time() - t0
print(f"Build time: {build_time_faiss:.4f} s, Query time: {query_time_faiss:.4f} s")

# -------------------------------
# HNSW
# -------------------------------
print("\n=== HNSW ===")
dim = corpus.shape[1]
num_elements = corpus.shape[0]

p = hnswlib.Index(space='l2', dim=dim)
t0 = time.time()
p.init_index(max_elements=num_elements, ef_construction=200, M=32)
p.add_items(corpus, np.arange(num_elements))
build_time_hnsw = time.time() - t0

p.set_ef(50)  # trade-off antara kecepatan dan akurasi
t0 = time.time()
idx_hnsw, dist_hnsw = p.knn_query(queries, k=k)
query_time_hnsw = time.time() - t0
print(f"Build time: {build_time_hnsw:.4f} s, Query time: {query_time_hnsw:.4f} s")

# -----------------------------------------
# 🔍 Evaluasi Recall@k
# -----------------------------------------
print("\n=== Recall@k Comparison ===")
rec_annoy = recall_at_k(idx_exact, idx_annoy, k)
rec_faiss = recall_at_k(idx_exact, idx_faiss, k)
rec_hnsw = recall_at_k(idx_exact, idx_hnsw, k)

print(f"Annoy Recall@{k}: {rec_annoy:.4f}")
print(f"FAISS Recall@{k}: {rec_faiss:.4f}")
print(f"HNSW Recall@{k}: {rec_hnsw:.4f}")

# -----------------------------------------
# 🧾 Summary Table
# -----------------------------------------
summary = pd.DataFrame({
    'Method': ['Exact NN', 'Annoy', 'FAISS', 'HNSWLIB'],
    'Build Time (s)': [build_time, build_time_annoy, build_time_faiss, build_time_hnsw],
    'Query Time (s)': [query_time, query_time_annoy, query_time_faiss, query_time_hnsw],
    f'Recall@{k}': [1.0, rec_annoy, rec_faiss, rec_hnsw]
})

print("\n=== SUMMARY ===")
print(summary.to_string(index=False))

=== Exact Nearest Neighbors (NN) ===
Build time: 7.4345 s, Query time: 0.0816 s

=== Annoy ===
Build time: 77.2724 s, Query time: 0.0260 s

=== FAISS ===
Build time: 0.0394 s, Query time: 0.3187 s

=== HNSW ===
Build time: 208.7090 s, Query time: 0.0053 s

=== Recall@k Comparison ===
Annoy Recall@10: 0.9853
FAISS Recall@10: 0.9926
HNSW Recall@10: 0.9937

=== SUMMARY ===
  Method  Build Time (s)  Query Time (s)  Recall@10
Exact NN        7.434536        0.081604   1.000000
   Annoy       77.272350        0.025984   0.985263
   FAISS        0.039402        0.318670   0.992632
 HNSWLIB      208.709033        0.005339   0.993684
